In [ ]:
train_data_dir = "train/images"
train_label_dir = "train/labels"
valid_data_dir = "valid/images"
valid_label_dir = "valid/labels"

train_data_filenames = [os.path.join(train_data_dir, f) for f in os.listdir(train_data_dir) if f.endswith('.jpg')]
train_label_filenames = [os.path.join(train_label_dir, f) for f in os.listdir(train_label_dir) if f.endswith('.txt')]

valid_data_filenames = [os.path.join(valid_data_dir, f) for f in os.listdir(valid_data_dir) if f.endswith('.jpg')]
valid_label_filenames = [os.path.join(valid_label_dir, f) for f in os.listdir(valid_label_dir) if f.endswith('.txt')]

train_dataset = tf.data.Dataset.from_tensor_slices((train_data_filenames, train_label_filenames))
train_dataset = train_dataset.map(lambda x, y: tuple(tf.py_function(
    read_image_and_annotation, [x, y], [tf.float32, tf.int32])), num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_data_filenames, valid_label_filenames))
valid_dataset = valid_dataset.map(lambda x, y: tuple(tf.py_function(
    read_image_and_annotation, [x, y], [tf.float32, tf.int32])), num_parallel_calls=tf.data.AUTOTUNE)
valid_dataset = valid_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet import ResNet101
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

# 데이터 경로 설정
train_dir = 'train/images/'
train_label_dir = 'train/labels/'
valid_dir = 'valid/images/'
valid_label_dir = 'valid/labels/'

# 데이터 증강을 위한 ImageDataGenerator 생성
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# 데이터셋 생성
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
        valid_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

# ResNet101 모델 불러오기
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델 구성
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(16, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=10, validation_data=valid_generator)

In [ ]:
import pandas as pd
import os

valid_images_path = 'valid/images'
valid_labels_path = 'valid/labels'

# 이미지 파일 경로와 라벨을 담은 데이터프레임 생성
valid_image_files = os.listdir(valid_images_path)
valid_labels_files = os.listdir(valid_labels_path)

valid_image_files = [os.path.join(valid_images_path, file) for file in valid_image_files]
valid_labels_files = [os.path.join(valid_labels_path, file) for file in valid_labels_files]

valid_df = pd.DataFrame({'filename': valid_image_files, 'label': valid_labels_files})

valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

In [ ]:
import os
import cv2
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split

train_dir = 'data/train/images/'
valid_dir = 'data/valid/images/'
train_label_dir = 'data/train/labels/'
valid_label_dir = 'data/valid/labels/'

# 이미지와 라벨을 저장할 리스트
x_train = []
y_train = []
x_valid = []
y_valid = []

# 이미지 크기 및 채널
img_size = (224, 224)
img_channel = 3

# train 데이터 로드
for filename in sorted(os.listdir(train_dir)):
    img_path = os.path.join(train_dir, filename)
    label_path = os.path.join(train_label_dir, filename[:-4] + '.txt')
    
    # 이미지와 라벨링 파일 불러오기
    img = cv2.imread(img_path)
    img = cv2.resize(img, img_size)  # 이미지 크기 조정
    with open(label_path) as f:
      label = int(f.readline().strip())
    
    # 이미지와 라벨을 리스트에 추가
    x_train.append(img)
    y_train.append(label)

# valid 데이터 로드
for filename in sorted(os.listdir(valid_dir)):
    img_path = os.path.join(valid_dir, filename)
    label_path = os.path.join(valid_label_dir, filename[:-4] + '.txt')
    
    # 이미지와 라벨링 파일 불러오기
    img = cv2.imread(img_path)
    img = cv2.resize(img, img_size)  # 이미지 크기 조정
    with open(label_path) as f:
      label = int(f.readline().strip())
    
    # 이미지와 라벨을 리스트에 추가
    x_valid.append(img)
    y_valid.append(label)

# NumPy 배열로 변환
x_train = np.array(x_train)
y_train = np.array(y_train)
x_valid = np.array(x_valid)
y_valid = np.array(y_valid)

# 데이터 전처리
x_train = np.array(x_train) / 255.
x_valid = np.array(x_valid) / 255.
y_train = keras.utils.to_categorical(y_train, num_classes=16)
y_valid = keras.utils.to_categorical(y_valid, num_classes=16)

# 모델 빌드
model = keras.applications.ResNet101(include_top=True, weights=None, input_shape=(224, 224, 3), classes=16)

# 모델 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_valid, y_valid))

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split

train_dir = 'data/train/images/'
valid_dir = 'data/valid/images/'
train_label_dir = 'data/train/labels/'
valid_label_dir = 'data/valid/labels/'

# train 데이터셋 불러오기
train_filenames = os.listdir(train_dir)
train_df = pd.DataFrame({'filename': train_filenames})
train_df['label'] = [int(open(train_label_dir+filename[:-4]+'.txt').readline().strip().split()[0]) for filename in train_filenames]

# validation 데이터셋 불러오기
valid_filenames = os.listdir(valid_dir)
valid_df = pd.DataFrame({'filename': valid_filenames})
valid_df['label'] = [int(open(valid_label_dir+filename[:-4]+'.txt').readline().strip().split()[0]) for filename in valid_filenames]

# 데이터 전처리
img_size = (224, 224)
batch_size = 32

train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col='filename',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_dataframe(
    valid_df,
    directory=valid_dir,
    x_col='filename',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# 모델 빌드
model = keras.applications.ResNet101(include_top=True, weights=None, input_shape=(224, 224, 3), classes=16)

# 모델 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_generator, epochs=10, batch_size=batch_size, validation_data=valid_generator)

전처리를 하는 방법에 따라 차이가 있습니다. 코드에서 보여지는 전처리 방법은 데이터를 모두 메모리에 불러와서 NumPy 배열로 변환한 후, 각각의 이미지와 레이블에 대해서 전처리를 진행한 뒤 다시 NumPy 배열로 변환하는 방법입니다. 이 방법은 데이터셋이 작을 때나, 이미지 개수가 적을 때에는 사용할 수 있지만, 큰 데이터셋의 경우에는 메모리를 많이 차지하여 out-of-memory (OOM) 에러가 발생할 수 있습니다.

반면, flow_from_dataframe() 함수는 데이터를 배치(batch) 단위로 처리하기 때문에 전체 데이터셋을 메모리에 불러오지 않아도 됩니다. 이 함수는 주어진 데이터프레임으로부터 이미지 경로와 레이블을 읽어와서 배치(batch) 단위로 이미지를 불러와 전처리를 수행합니다. 이 방법은 데이터셋의 크기가 클 때 효율적으로 처리할 수 있습니다.